Data is being pulled automatically from RKStudio server and then saved to `3_3_2_processed_data` folder.


**PROCESS**
1. `launchd` runs the `.plist` file `/Library/LaunchAgents/thesis.djw.pull-rkstudio-data.plist` every day at 10:40am
2. `thesis.djw.pull-rkstudio-data.plist` runs `.sh` script `/download/download_rkstudio_data.sh`
3. `download_rkstudio_data.sh` runs `.py` script `/download/download_rkstudio_data.py`
4. `download_rkstudio_data.py` downloads any **new** files from RK Studio, and saves to `.pkl` in `3_3_2_processed_data` folder.

---

**Note:** 
- The `create_dict_df()` function in `download_rkstudio_data.py` is inefficient as it recreates the file from scratch each time it runs instead of just appending new data. If things get slow this could be a spot to address.
- The `create_dict_df()` function is also simply replacing previous file with current file

In [6]:
save_path = "/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_2_processed_data/"

in_file = open(save_path + "app_data.pkl", "rb")
app_data = pickle.load(in_file)

In [2]:
import zipfile
import glob
import os
import pandas as pd
import pickle

In [3]:
# path to RKStudio exported data
file_path = "/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_1_raw_data/testing"

Something wierd happened where we started getting additional files named like:

`SurveyData/01801252-3a7e-4f5f-8b6d-49e8da3902f3/0b3181d1-4bb8-eb11-aaaa-0afb9334277d/reactionTime/DeviceMotion`

These seemed perhaps to be `.json`? For example:

```
Index(['{"items":[{"attitude":{"y":-0.015030415521581996032',
       'w:0.9316986828576702464', 'z:0.00000000000000011102230246251565056',
       'x:0.362920855519224064}', 'timestamp:3568168.594124125184',
       'rotationRate:{"x":0.03503423929214477312', 'y:-0.04006228968501091328',
       'z:0.0013012606650590896128}',
       'userAcceleration:{"x":-0.002143738791346550016',
       'y:-0.0015905499458312988672',
       ...
       'z:-0.4993825852870940672}',
       'userAcceleration:{"x":-0.2922154068946837504',
       'y:0.2403054237365722112', 'z:-0.389307737350464}',
       'gravity:{"x":0.09684188663959502848', 'y:-0.7316384911537170432',
       'z:-0.6747791767120361472}', 'magneticField:{"y":0.884', 'z:0.885',
       'x:0.884'],
      dtype='object', length=30590)
```

I am assuming this is an error and there isn't anything meaningful here (we already have a score for the **Reaction Time** task), so am removing from the `app_data` dataframe.

In [14]:
def create_df_dict(file_path, save_path):
    # createstartswithty dictionary
    app_data = {}

    # create list of zip files
    data_files = glob.glob(f'{file_path}/*.zip')
    data_files.sort(reverse=True)
    # move the first entry to the end as it has slighlty different syntax
    data_files = data_files[1:]+data_files[0:1]

    # extract all csv files from zip
    for zip_file in data_files:
        zf = zipfile.ZipFile(zip_file)

        # list of .csv files in zip
        file_names = zf.namelist()

        # loop through .csv files
        for file in file_names:
                # added because there seemed to be some weird files being included
                if not file.startswith('SurveyData/'):
                    try:
                        df = pd.read_csv(zf.open(file), parse_dates=True, low_memory=False)
                        # clean up filename to use as key
                        sep = '_'
                        file = file.replace('.', '_')
                        file = file.split(sep,1)[0]

                    # some files are sometimes empty (e.g. notifications)
                    except pd.errors.EmptyDataError:
                        continue # will skip the rest of the block and move to next file

                    # append each day to df
                    try:
                        app_data[file] = app_data[file].append(df, ignore_index=True)
                    except KeyError:
                        app_data[file] = df

In [14]:
app_data = create_df_dict(file_path)

In [22]:
save_path = "/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_2_processed_data/"

out_file = open(save_path + "app_data.pkl", "wb")
pickle.dump(app_data, out_file)
out_file.close()

In [23]:
in_file = open(save_path + "app_data.pkl", "rb")
app_data = pickle.load(in_file)